In [46]:
import numpy as np
from numpy.random import normal, seed
from numpy import exp, log, sqrt
from scipy.stats import norm
import matplotlib.pyplot as plt

In [252]:
S0 = 100
K = 92
r = 0.06
sigma = 0.23
T = 150 / 365

** The Black-Scholes Formula for European Call Options**
$$
    S_0 \Phi(d_+) - K e^{-rT} \Phi(d_-)
$$

Where,
- $d_+ = \frac{1}{\sigma\sqrt T}\left(\log \frac{S_0}{K}+ (r + \frac{1}{2}\sigma^2)T\right)$
- $d_- = \frac{1}{\sigma\sqrt T}\left(\log \frac{S_0}{K}+ (r - \frac{1}{2}\sigma^2)T\right) = d_+ - \sigma\sqrt T$

In [289]:
def european_option(s, k, sigma, rate, ttm):
    """
    Parameters
    ----------
    s: float
        The value of the underlying at t=0
    k: float
        The value of the strike
    sigma: float
        Volatility of the underlying
    rate: float
        The risk-free interest rate
    ttm: float
        The time to maturity –in years– to maturity.
    """
    d1 = (log(s / k) + (r + sigma ** 2 / 2) * ttm) / (sigma * sqrt(ttm))
    d2 = d1 - sigma * sqrt(ttm)
    return s * norm.cdf(d1) - k * exp(-rate * ttm) * norm.cdf(d2)

In [298]:
european_option(S0, K, sigma, r, T)

12.125338828163677

** Simulating payoff **

Consider
$$
    S_t = S_0e^{\sigma W_t + (r - \frac{1}{2}\sigma^2)t}
$$

In [301]:
def mean_price(S0, K, T, r, sigma, nsimul):
    sum_price = 0
    for _ in range(nsimul):
        WT = normal(scale=sqrt(T))
        ST = S0 * exp(sigma * WT + (r - sigma ** 2 / 2) * T)
        payoff = max(ST - K, 0)
        sum_price += payoff
    return sum_price / nsimul

price = 0
nsim = 1500
for _ in range(nsim):
    price += mean_price(S0, K, T, r, sigma, nsim)
    
price = exp(-r * T) * price / nsim
price

12.114696136969998